# Pythonで線形計画問題(+混合整数計画問題)を解く

## 最適化問題の特徴

* 実世界でよくあらわれる問題
例　: 
    * 利益が最大となる生産計画は？
    * 最も余裕のある月間のアルバイトのシフトは？
    * 最も無駄がない乗り換えルートは？


* 問題の種類に応じて、様々な手法がある
    * 目的は１つ？複数？(安くて時間の短い乗り換えルート⇒2目的)
    * 変数は連続値？離散値？ラベル?(経路の総和⇒連続値, 通る駅の順番⇒ラベル)
    * 答えにはどんな条件がある？(コストは1000円以内)
    
　　⇒ 同種の問題でも複数の解き方がある
  
    ⇒ 手法の選択が難しい


* パラメータが増えると現実時間ですべての答えを計算するのが難しくなる

$ 1! = 1$

$10! = 3,628,800$

$20! = 2,432,902,008,176,640,000$

* 条件が少し変わっただけで、解き方が格段に難しくなる(目的関数、制約条件)

簡単   : $f(x) = 10 * x_1 + 20 * x_2 + 40 * x_3$

難しい : $f(x) = 10 * x_1^2 + 20 * x_2 + 40 * x_3$


## pulpとは
線形計画問題を解く Python パッケージ

## 線形計画問題とは
目的関数と制約条件が１次式で表される最適化問題

例
* コストと利益が異なるいくつかの製品を、決まった数量作るときに、利益が最大になるような量の組み合わせを見つける
* パズル(例えば数独とか)を解く


## どうやって解く？
### 自分で解き方を考える(実装する)
1. 問題の定式化
1. 問題の種類に合わせて手法を検討
1. 必要ならば問題を緩和(条件を緩める等)
1. (近傍を定義)
1. 実装
1. 評価・再検討(パフォーマンスに納得がいかなければ2.に戻る)

⇒ 結構大変
* かなりの数の手法がある
* 研究的に反復しながら進めることが多い
    * 問題の定式化・緩和方法、近傍の定義、選択したアルゴリズム、実装方法すべてに依存して性能が決まるので、かなりの経験が必要

### 問題に合わせて専用のソルバーを使う
1. 問題の定式化
1. 問題の種類に合わせてどのソルバーを使うか検討
1. 必要ならば問題を緩和(条件を緩める等)
1. ソルバーを使って問題を解く
1. 評価・再検討(パフォーマンスに納得がいかなければ2.に戻る)

⇒ 自前実装よりかは楽
* 問題の定式化や緩和部分に注力できる
* 大体がPythonのAPIを持っているので、実装作業が楽
* ソルバー間で共通の問題を記述する形式(LP形式)がある ⇒ 覚えることが比較的少なくて済みそう
* 無償ソルバーよりも商用ソルバーの方が圧倒的に性能が良い場合が多いので、お金がかかる


# インストール方法
pipでインストールできる
> sudo pip install pulp

# 例1 : 簡単な例

$Min$ : $f \left( a, b \right) = a + b$

$Subject \ to$ : 
$0 <= a \\
0.1<= b \\
a + b = 0.5
$

In [ ]:
import pulp

# 問題オブジェクト(目的関数)の作成
problem = pulp.LpProblem('f1',pulp.LpMinimize)

# 変数の設定
a = pulp.LpVariable('a',0,1)
b = pulp.LpVariable('b',0,1)

# 目的関数の設定
problem += a + b

# 制約条件の設定
problem += a >= 0
problem += b >= 0.1
problem += a + 2 * b == 0.5

# 問題表示
print(problem)

# ソルバー実行
status = problem.solve()

# 答え表示
print(pulp.LpStatus[status])
print('a : ' + str(a.value()))
print('b : ' + str(b.value()))
print(problem)

# 例2 輸送最適化問題
* 倉庫群から工場群へ部品を搬送したい。輸送費が最小となる計画を求めたい。
    * 2つの工場$A_1$,$A_2$で製品を生産
    * 3つの取引先$B_1$,$B_2$,$B_3$に納入
    * 輸送コストを最小にしたい


* 各工場の生産量

|$A_1$|$A_2$|
|-----|-----|
|90|80|

* 各取引先の注文量

|$B_1$|$B_2$|$B_3$|
|-----|-----|-----|
|70|40|60|

* 輸送コスト

||$B_1$|$B_2$|$B_3$|
|-----|-----|-----|-----|
|$A_1$| 4| 7|12|
|$A_2$|11| 6| 3|

http://qiita.com/Tsutomu-KKE@github/items/070ca9cb37c6b2b492f0

## 式にしてみる
* 変数の設定

工場$A_i$から取引先$B_j$への輸送量:$x_{ij}$

* 目的関数の設定

$Min$ : $f \left( x \right) = 4x_{11} + 7x_{12} + 12x_{13} + 11x_{21} + 6x_{22} + 3x_{23}$

* 工場での生産量に関する条件

$x_{11} + x_{12} + x_{13} <= 90$,
$x_{21} + x_{22} + x_{23} <= 80$

* 取引先への注文量に関する条件

$x_{11} + x_{21} = 70$,
$x_{12} + x_{22} = 40$,
$x_{13} + x_{23} = 60$

* 輸送量に関する非負条件

$x_{ij} >= 0$

In [ ]:
import pulp

# 問題オブジェクト(目的関数)の作成
problem = pulp.LpProblem('f2',pulp.LpMinimize)

# 変数の設定
rows = range(1,3)
cols = range(1,4)
c = pulp.LpVariable.dict('c',(rows,cols),0,1000,"Continuous")
x = pulp.LpVariable.dict('x',(rows,cols),0,1000,"Continuous")

c[1,1], c[1,2], c[1,3], c[2,1], c[2,2], c[2,3] = 4, 7, 12, 11, 6, 3

# 目的関数の設定
problem += c[1,1] * x[1,1] + c[1,2] * x[1,2] + c[1,3] * x[1,3] + c[2,1] * x[2,1] + c[2,2] * x[2,2] + c[2,3] * x[2,3]

# 制約条件の設定
# 工場での生産量に関する条件
problem += x[1,1] + x[1,2] + x[1,3] <= 90
problem += x[2,1] + x[2,2] + x[2,3] <= 80

# 取引先への注文量に関する条件
problem += x[1,1] + x[2,1] == 70
problem += x[1,2] + x[2,2] == 40
problem += x[1,3] + x[2,3] == 60

# 輸送量に関する非負条件
problem += x[1,1] >= 0
problem += x[1,2] >= 0
problem += x[1,3] >= 0
problem += x[2,1] >= 0
problem += x[2,2] >= 0
problem += x[2,3] >= 0

# 問題表示
print(problem)

# # ソルバー実行
status = problem.solve()

# 答え表示
print("ステータス : ",pulp.LpStatus[status])
print("解:")
for i in x.values():
    print(i, ":", i.value())

print("計算時間 :" + str(problem.solutionTime))


## 例3 数独

|05|03|  |  |07|  |  |  |  |
|--|--|--|--|--|--|--|--|--|
|06|  |  |01|09|05|  |  |  |
|  |09|08|  |  |  |  |06|  |
|08|  |  |  |06|  |  |  |03|
|04|  |  |08|  |03|  |  |01|
|07|  |  |  |02|  |  |  |06|
|  |06|  |  |  |  |02|08|  |
|  |  |  |04|01|09|  |  |05|
|  |  |  |  |08|  |  |07|09|

* 変数の設定
9×9マス、入る値が1-9の9種類
→ 9×9×9個の変数を準備(値は0または1となる)

例) 縦x横yのマスの数字がvのとき、

$c_{x,y,v} = 1 \ $,
$c_{x,y,v'} = 0$($v' \neq v$)

* 目的関数の設定


* 制約条件の設定
    * 1つのマスに入る値は 1つだけ

$ c_{1,y,v} + c_{2,y,v} + ・・・ + c_{1,y,v} = 1 \ $

    * 既に数字が埋められているときは必ずその数字のみがマスに入る

    * 同じ列・行・ボックスには同じ数字は入らない
    * 行の制約
    * 列の制約
    * ボックスの制約


In [ ]:
import pulp

# 問題オブジェクト(目的関数)の作成
problem = pulp.LpProblem('Sudoku', pulp.LpMinimize)

# 変数の設定
numbers = range(1, 10)
xs = range(1, 10)
ys = range(1, 10)
choices = pulp.LpVariable.dicts("Choice",(xs, ys, numbers),0,1,pulp.LpInteger)

# 目的関数の設定
board = """
530070000
600195000
098000060
800060003
400803001
700020006
060000280
000419005
000080079"""

print(board)
width = len(board.strip().split("\n")[0])
height = len(board.strip().split("\n"))
board = [map(int, list(line)) for line in board.strip().split("\n")]

problem += 0

for x in range(width):
    for y in range(height):
        if board[y][x] > 0:
            problem += choices[board[y][x]][x+1][y+1] == 1

# 制約条件の設定
for y in ys:
    for x in xs:
        problem += pulp.lpSum([choices[v][x][y] for v in numbers]) == 1, ""

for v in numbers:
    for y in ys:
        problem += pulp.lpSum([choices[v][x][y] for x in xs]) == 1

    for x in xs:
        problem += pulp.lpSum([choices[v][x][y] for y in ys]) == 1

    for x in [1, 4, 7]:
        vs = []
        for y in [1, 4, 7]:
            print([[choices[v][x+i][y+j] for i in range(3) for j in range(3)]])
            prob += pulp.lpSum([[choices[v][x+i][y+j] for i in range(3) for j in range(3)]]) == 1      

# ソルバー実行
status = problem.solve()

# 問題表示
print(problem)

# 答え表示
print("ステータス : ",pulp.LpStatus[status])
print("解:")
for y in ys:
    for x in xs:
        for v in numbers:
            if choices[v][x][y].value() == 1:
                print(v)

for i in x.values():
    print(i, ":", i.value())

print("計算時間 :" + str(problem.solutionTime))


## 混合整数計画問題について
* 世の中の問題の多くは混合整数計画問題
    * 線形計画問題に比べると難しい問題クラス

* 最近のソルバーの性能向上が目覚ましい領域

* gurobiが鉄板
    * 利用例 : ブレインパッドのリスティング広告最適化ツール「L2Mixer」
    * 商用ソフト
    * 問題を公開すればNEOSサーバで無料で使える
gurobiにお布施する？NEOSサーバ使う？

# 問題とソルバー・ライブラリの関係
* 線形計画問題 ⇒ Pulp
* 混合線形計画問題 ⇒ gurobi
* ネットワーク最適化 ⇒ pythonのnetworkxパッケージ
* 多目的最適化 ⇒ MOEA
* その他
    * juliaopt